In [ ]:
# Import necessary libraries
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load and preprocess images
def load_and_preprocess_image(image_path, target_size=(256, 256)):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  
    return img


dataset_path = 'path/to/your/dataset'


# Example: loading two images for content and style
content_image_path = 'path/to/content/image.jpg'
style_image_path = 'path/to/style/image.jpg'

content_image = load_and_preprocess_image(content_image_path)
style_image = load_and_preprocess_image(style_image_path)


In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model

def create_model():
    base_model = VGG19(weights='imagenet', include_top=False)
    base_model.trainable = False

    # Extract feature maps from intermediate layers
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

    content_outputs = [base_model.get_layer(layer).output for layer in content_layers]
    style_outputs = [base_model.get_layer(layer).output for layer in style_layers]

    model_outputs = content_outputs + style_outputs

    # Create the model
    model = Model(inputs=base_model.input, outputs=model_outputs)
    return model


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

def create_loss_model(content_weight, style_weight):
    model = create_model()

    # Freeze the layers
    for layer in model.layers:
        layer.trainable = False

    # Define content and style loss functions
    content_loss = MeanSquaredError()
    style_loss = MeanSquaredError()

    # Specify layer weights for content and style
    content_weight /= len(content_layers)
    style_weight /= len(style_layers)

    # Compile the model
    model.compile(optimizer=Adam(), loss=[content_loss] * len(content_layers) + [style_loss] * len(style_layers))

    return model

# Example usage
content_weight = 1e3  
style_weight = 1e-2  
model = create_loss_model(content_weight, style_weight)
model.fit(x=[content_image, style_image], y=[content_image, style_image], epochs=10, batch_size=1, verbose=1)


In [ ]:
def stylize_image(input_image, model):
    input_image = load_and_preprocess_image(input_image)
    stylized_features = model.predict(input_image)

    # Extract stylized content and style features
    stylized_content = stylized_features[:len(content_layers)]
    stylized_style = stylized_features[len(content_layers):]

    

    return stylized_image
